# 高级分词、词干提取与词形还原

如前所述，CountVectorizer和TfidfVectorizer 中的特征提取相对简单，还有更为复杂的方法。在更加复杂的文本处理应用中，通常需要改进的步骤是词袋模型的第一步:分词(tokenization)。这一步骤为特征提取定义了一个单词是如何构成的。

我们前面看到，词表中通常同时包含某些单词的单数形式和复数形式，比如"drawback"和"drawbacks"、"drawer"和“drawers"、"drawing"和"drawings"。对于词袋模型而言"drawback"和"drawbacks"的语义非常接近，区分二者只会增加过拟合，并导致模型无法充分利用训练数据。同样我们还发现、词表中包含像 "replace"、"replaced"、"replacement"、"replaces"和"replacing"这样的单词，它们都是动词"to replace"的不同动词形式或相关名词。与名词的单复数形式一样，将不同的动词形式及相关单词视为不同的词例，这不利于构建具有良好泛化性能的模型

这个问题可以通过用**词干**(word stem) 表示每个单词来解决，这一方法涉及找出|或**合并**(conflate)|所有具有相同词干的单词。如果使用基于规则的启发法来实现 (比如删除常见的后缀)，那么通常将其称为**词干提取**(stemming)。如果使用的是由已知单词形式组成的字典 (明确的且经过人工验证的系统)，并且考虑了单词在句子中的作用，那么这个过程被称为**词形还原**(lemmatization)，单词的标准化形式被称为**词元**(lemma)。词干提取和词形还原这两种处理方法都是**标准化**(normalization)的形式之一，标准化是指尝试提取，个单词的某种标准形式。标准化的另一个有趣的例子是拼写校正，这种方法在实践申很有用。

为了更好地理解标准化，我们来对比一种词干提取方法(Porter词干提取器，一种广泛使用的启发法集合，从nltk 包导入)与spacy 包中实现的词形还原:

In [1]:
import nltk

# 加载spacy的英语模型
import spacy
en_nlp = spacy.load("en_core_web_sm")

# 将nltk的Porter词干提取器实例化

stemmer = nltk.stem.PorterStemmer()

# 定义一个函数来对比spacy中的词形还原与nltk的词干提取
def compare_normalization(doc):
    # 在spacy中对文档进行分词
    doc_spacy = en_nlp(doc)
    # 打印spacy找到的词元
    print("Lemmatization:")
    print([token.lemma_ for token in doc_spacy])
    # 打印Porter词干提取器找到的词例
    print("Stemming:")
    print([stemmer.stem(token.norm_.lower()) for token in doc_spacy])

我们将用一个句子来比较词形还原与 Porter 词干提取器，以显示二者的一些区别:

In [2]:
compare_normalization(u"Our meeting today was worse than yesterday, "
                       "I'm scared of meeting the clients tomorrow.")

Lemmatization:
['our', 'meeting', 'today', 'be', 'bad', 'than', 'yesterday', ',', 'I', 'be', 'scared', 'of', 'meet', 'the', 'client', 'tomorrow', '.']
Stemming:
['our', 'meet', 'today', 'wa', 'wors', 'than', 'yesterday', ',', 'i', 'am', 'scare', 'of', 'meet', 'the', 'client', 'tomorrow', '.']


词干提取总是局限于将单词简化成词干，因此“was”变成了“wa"，而词形还原可以得到正确的动词基本词形“be"。同样，词形还原可以将“worse”标准化为"bad"，而词干提取得到的是“wors"。另一个主要区别在于，词干提取将两处"meeting"都简化为“meet"。利用词形还原，第一处"meeting"被认为是名词，所以没有变化，而第二处meeting"被认为是动词，所以变为“meet"。一般来说，词形还原是一个比词干提取更复杂的过程，但用于机器学习的词例标准化时通常可以给出比词干提取更好的结果。

虽然scikit-learn 没有实现这两种形式的标准化，但 CountVectorizer 允许使用 tokenizer参数来指定使用你自己的分词器将每个文档转换为词例列表。我们可以使用 spacy 的词形还原了创建一个可调用对象，它接受一个字符串并生成一个词元列表:

In [3]:
from sklearn.datasets import load_files
import numpy as np
reviews_train = load_files("E:/clone/machine-learning/data/aclImdb/train/")
# load_files返回一个Bunch对象，其中包含训练文本和训练标签

data_array = np.array(reviews_train.data)
target_array = np.array(reviews_train.target)

# Filter out documents where the target is not equal to 2
labeled_indices = np.where(target_array != 2)[0]
text_train, y_train = data_array[labeled_indices], target_array[labeled_indices]
text_train = [doc.replace(b'<br />',b' ') for doc in text_train]

reviews_test = load_files("E:/clone/machine-learning/data/aclImdb/test/")
data_array = np.array(reviews_test.data)
target_array = np.array(reviews_test.target)
text_test, y_test = data_array, target_array
text_test = [doc.replace(b'<br />',b' ') for doc in text_test]

In [4]:
# 技术细节:我们希望使用由CountVectorizer所使用的基于正则表达式的分词器
# 并仅使用spacy的词形还原。
# 为此，我们将en_nlp.tokenizer (spacy分词器) 替换为基于正则表达式的分词
# with the regexp based tokenization
import re
import spacy
from sklearn.feature_extraction.text import CountVectorizer

# 在CountVectorizer中使用的正则表达式
regexp = re.compile('(?u)\\b\\w\\w+\\b')

# 加载spacy语言模型
en_nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

# 使用 SpaCy 文档处理管道创建自定义标记器
def custom_tokenizer(document):
    tokens = regexp.findall(document)
    return [token.lemma_ for token in en_nlp(" ".join(tokens))]

# 利用自定义分词器来定义一个计数向量器
lemma_vect = CountVectorizer(tokenizer=custom_tokenizer, min_df=5)

#利用带司形还原的CountVectorizer对text train进行变换
X_train_lemma = lemma_vect.fit_transform(text_train)
print("X_train_lemma.shape: {}".format(X_train_lemma.shape))

# 标准的CountVectorizer，以供参考
vect = CountVectorizer(min_df=5).fit(text_train)
X_train = vect.transform(text_train)
print("X_train.shape: {}".format(X_train.shape))


D:\python\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


X_train_lemma.shape: (25000, 21580)
X_train.shape: (25000, 27271)


从输出中可以看出，词形还原将特征数量从27271个(标准的CountVectorizer 处理过程)减少到 21580个。词形还原可以被看作是一种正则化，因为它合并了某些特征。因此我们预计，数据集很小时词形还原对性能的提升最大。为了说明词形还原的作用，我们将使用stratifiedShuffleSplit 做交叉验证，仅使用1%的数据作为训练数据，其余数据作为测试数据:

In [5]:
# 仅使用1%的数据作为训练集来构建网格搜索
from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV
from sklearn.linear_model import LogisticRegression

param_grid = {'C': [0.001, 0.01, 0.1, 1, 10]}
cv = StratifiedShuffleSplit(n_splits=5, test_size=0.99,
                            train_size=0.01, random_state=0)
grid = GridSearchCV(LogisticRegression(max_iter=1000), param_grid, cv=cv)
#利用标谁的CountVectorizer进行网格搜索
grid.fit(X_train, y_train)
print("Best cross-validation score "
      "(standard CountVectorizer): {:.3f}".format(grid.best_score_))
#利用词形还原进行网格搜索
grid.fit(X_train_lemma, y_train)
print("Best cross-validation score "
      "(lemmatization): {:.3f}".format(grid.best_score_))

Best cross-validation score (standard CountVectorizer): 0.718
Best cross-validation score (lemmatization): 0.726


在这个例子中，词形还原对性能有较小的提高。与许多特征提取技术一样，其结果因数据集的不同而不同。词形还原与词干提取有时有助于构建更好的模型(或至少是更简洁的模型)，所以我们建议你，在特定任务中努力提升最后一点性能时可以尝试下这些技术。